# Tracers and barriers paper

Tracers and Barriers paper

This script is for the generation of flow model input files, running the flow model, and the generation of transport model input files.

THEN --> run the transport model.
THEN --> post-process the data using the post-processing (TracersBarriers_01_PP.py) script.

## Case Study (CS) & recharge descriptions
CS1 is with no barrier.
CS2 is fully-penetrating barrier.
CS3 is buried barrier.
CS4 is fully penetrating barrier, oblique across part of the aquifer.

Recharge 1 = Diffuse recharge across the whole aquifer.
Recharge 2 = Upgradient (mountain-front) recharge only.

Models A, B, C = No barrier
Models D, E, F = Barrier

Models A, D = Recharge 1
Models B, E = Recharge 2
Models C, F = Recharge 3

## Importing packages

In [20]:
import numpy as np
import matplotlib as mpl
import flopy
from flopy import mt3d
import pandas as pd
import subprocess
import os
import time
import sys
import datetime
from decimal import Decimal

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('pandas version: {}'.format(pd.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.18.1
matplotlib version: 3.1.3
pandas version: 1.0.2
flopy version: 3.3.0


## Setting up directories

In [21]:
modelName    = "TB_01_a"
modelName_mt = "%s_mt3d"%modelName

print("Flow model name is: %s, transport model name is: %s" %(modelName, modelName_mt))

MODFLOW_folder      = r'C:\Program Files\MODFLOW-NWT_1.1.3\bin\MODFLOW-NWT_64'

project_folder      = r'C:\workspace\Proj2_TracersBarriers/'
MT3D_USGS_folder    = r'C:\Program Files\mt3d-usgs_Distribution\bin\MT3D-USGS_64'
exe_name            = r'C:\workspace\Proj2_TracersBarriers\MT3D_Reaction_sngl' 
mt3d_version        = 'mt3d-usgs' 

this_model_folder = project_folder + modelName
if not os.path.exists(this_model_folder):
    os.makedirs(this_model_folder)

dataDirectory = this_model_folder + '/Data'
if not os.path.exists(dataDirectory):
    os.makedirs(dataDirectory)

figureDirectory = this_model_folder + '/Figures'
if not os.path.exists(figureDirectory):
    os.makedirs(figureDirectory)

os.chdir(this_model_folder)

print("Current working directory is: " + str(os.getcwd()))

Flow model name is: TB_01_a, transport model name is: TB_01_a_mt3d
Current working directory is: C:\workspace\Proj2_TracersBarriers\TB_01_a


## Model discretisation

In [22]:
nlay = 12                    # Number of layers.
Lx = 10000.                  # Length of the model sides in metres.
Ly = 5000                    # Rectangular domain.
delr = 25.                   # Spacing length across rows in metres.
delc = delr                  # The spacing along columns and rows are equal.
ncol = int(Lx/delr)          # Number of columns.
nrow = int(Ly/delc)          # Number of rows.
ncells = nlay*ncol*nrow      # Total number of cells in the model.
ncells_per_layer = ncol*nrow # Number of cells per layer.
surface_area = Lx*Ly         # m^2

ztop = 300.                  # Top of model.
zbot = 0.                    # Base of model.
delv = (ztop - zbot) / nlay  # Length of cells in z direction.
botm = np.linspace(ztop, zbot, nlay + 1) 

## Hydraulic properties

In [23]:
sHead = 290                  # Starting head across aquifer.

sy = 0.1                     # Specific yield.
ss = sy/ztop                 # Specific storage.
laytyp = 1                   # Layer type (1 = convertible; 0 = confined).

hk_aquifer = 1.              # Hydraulic conductvity along rows (m/day).
vka = hk_aquifer/10.         # Vertical hydraulic conductivity.

prsity = sy
tortuosity_factor = 0.09

recharge_flux = 1.3699e-5    # m/d This is equivalent to 5 mm per year.
total_recharge = recharge_flux*surface_area # m^3 per day.
total_recharge_per_y = total_recharge*365.25 # m^3 per year.

length_simulation = 1        # Length of the steady-state flow model (days). 

## Barrier characteristics

In [24]:
hk_barrier           = hk_aquifer/1000 # m/d 3 orders of magnitude lower than the aquifer.
barrier_width        = 50 # m
number_barrier_cols  = barrier_width/delr  # So we can test/change the grid spacing.
column_of_barrier    = []

if number_barrier_cols == 1:
    column_of_barrier.append(int(ncol/2)) 
elif number_barrier_cols%2 == 0: # Even number
    for i in range(int(number_barrier_cols/2)):
        a = int(number_barrier_cols/2) - i
        column_of_barrier.append(int(ncol/2)-a)

    for i in range(int(number_barrier_cols/2)):
        print(i)
        column_of_barrier.append(int(ncol/2)+i)
else: # Odd number        
    for i in range(int(number_barrier_cols/2)):
        a = int(number_barrier_cols/2) - i
        column_of_barrier.append(int(ncol/2)-a)

    for i in range(int(number_barrier_cols/2)):
        print(i)
        column_of_barrier.append(int(ncol/2)+i)

    column_of_barrier.append(int(ncol/2)+int(number_barrier_cols/2))
    
percentage_gap        = 10 # CS2 
first_row_barr        = int(nrow/percentage_gap) # CS2 
last_row_barr         = int(nrow-(nrow/percentage_gap)) # CS2 
start_lay_barr        = int(nlay/2) # CS3
end_lay_barr          = nlay # CS3

0


## Making the hydraulic conductivity arrays

In [25]:
number_of_casestudies = 3

Case Study 1: No barrier

In [26]:
hk_array_1 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)      

Case Study 2: Fully-penetrating barrier

In [27]:
hk_array_2 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)     
  
for il in range(nlay):
    for ir in range(first_row_barr, last_row_barr, 1): 
        for col_number in range(len(column_of_barrier)):
            hk_array_2[il, ir, column_of_barrier[col_number]] = hk_barrier 

Case Study 3: Buried barrier

In [28]:
hk_array_3 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)     

for il in range(start_lay_barr, end_lay_barr):
    for ir in range(nrow): 
        for col_number in range(len(column_of_barrier)):
            hk_array_3[il, ir, column_of_barrier[col_number]] = hk_barrier 

## Recharge characteristics

In [29]:
number_of_recharge_scenarios = 2

Recharge array 1 = diffuse recharge across the whole aquifer in equal amounts.

In [30]:
recharge_array_1 = recharge_flux * np.ones((nrow, ncol), dtype=np.float32)

Recharge array 2 = recharge across only the top of the catchment.
Recharge across all rows, across first 50 columns (One quarter of catchment).

In [31]:
n_cols_rech_2    = int(ncol/4)
n_cells_rech_2   = int(n_cols_rech_2*nrow)
SA_recharge_2    = Ly*n_cols_rech_2*delc # Surface area (m^2)
recharge_flux_2  = total_recharge/SA_recharge_2 # m/d

recharge_array_2 = np.zeros((nrow, ncol), dtype=np.float32)

for c in range(n_cols_rech_2):  # Applying recharge to every row to the columns specified.
    for r in range(nrow): 
        recharge_array_2[r, c] = recharge_flux_2

## Boundary conditions

General head boundary on the right-hand side.
No boundary on the left-hand side so use default condition which is a no-flow boundary.

In [32]:
stageleft  = float(sHead)
stageright = float(sHead)

bound_sp1  = []
for il in range(nlay):

    condright = (hk_aquifer * (stageright - zbot) * delc)/10

    for ir in range(nrow):
        bound_sp1.append([il, ir, ncol - 1, stageright, condright])

print('Adding ', len(bound_sp1), 'GHBs for stress period 1.')

ghb_spd = {0: bound_sp1}    

Adding  2400 GHBs for stress period 1.


Specifying starting heads

In [33]:
if nlay == 1:
    ibound = np.ones((nrow, ncol), dtype=np.int32)
    strt = sHead * np.ones((nrow, ncol), dtype=np.float32)

else:
    ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
    strt = sHead * np.ones((nlay, nrow, ncol), dtype=np.float32) 
   

## Stress periods

1 Stress period: steady state only 

In [34]:
nper   = 1 # Number of model stress periods.
perlen = [length_simulation] 

nstp   = [1] # Number of time steps in each stress period.
tsmult = [1]
steady = [True] # True = steady state.

print("One steady-state stress periods set up")

One steady-state stress periods set up


## Input files and running flow model

Running multiple model scenarios:
         
* 1st:   Different recharge scenarios.
     
* 2nd:   Different case studies, representing the different hydraulic conductivities (barrier configurations).


In [35]:
print("The total number of scenarios is: " + str(
                        number_of_recharge_scenarios*number_of_casestudies))

The total number of scenarios is: 6


If I were to do this again, I would not do it as a loop. But it is the way that I did it. Each loop runs a different model then I save the output.


This cell sets up the loop - I can get rid of it.

In [ ]:
t1 = time.time()  
for rechargeScenario in range(number_of_recharge_scenarios): # number_of_recharge_scenarios
    print("Recharge scenario: " + str(rechargeScenario+1))
    
    for caseStudy in range(number_of_casestudies): # number_of_casestudies
        print("Case study: " + str(caseStudy+1))
        
        

## Function to set up and run the flow model

Need to be careful whether I am adding 1 to recharge scenario and case study or not, I have changed things around a bit. 

In [ ]:
def flow_model(rechargeScenario, caseStudy):
    
    modelname = str(modelName) + "_R" + str(rechargeScenario) + "_CS" + str(caseStudy)
    print("Flow model name: " + str(modelname))
        
    modelname_mt = modelName_mt3d + "_R" + str(rechargeScenario+1) + "_CS" + str(caseStudy+1)
    print("Transport model name: " + str(modelname_mt))
    
    mf = flopy.modflow.Modflow(modelname, exe_name=MODFLOW_folder, version='mfnwt') # MODFLOW-NWT
    
    dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, 
                                        delc=delc,top=ztop, botm=botm[1:],
                                        tsmult = tsmult, nper=nper, perlen=perlen, 
                                        nstp=nstp, steady=steady)
        
    print("Discretisation module set up")
       
    # Which hydraulic conductivity array do I use, depending on which scenario is applied:
        
    if (caseStudy+1) == 1:
        hk_array = hk_array_1
            
    elif(caseStudy+1) == 2:
        hk_array = hk_array_2
                   
    else:
        hk_array = hk_array_3
            
    uwp = flopy.modflow.mfupw.ModflowUpw(mf, hk=hk_array, vka=vka, sy=sy, ss=ss, ipakcb=53, iphdry = 53,
                                                 laytyp=laytyp) # MODFLOW- NWT
    print("Upstream weighting package set up")
        
    # --- Solver --- #
            
    nwt = flopy.modflow.mfnwt.ModflowNwt(mf, headtol=1e-05, maxiterout=200, 
                        thickfact=1e-07, linmeth=2, iprnwt=1, ibotav=1, options='SPECIFIED', 
                        Continue=True, dbdtheta=0.9, dbdkappa=1e-05, dbdgamma=0.0, momfact=0.1, 
                        backflag=1, maxbackiter=30, backtol=1.05, backreduce=0.9, maxitinner=50, 
                        ilumethod=2, levfill=5, stoptol=1e-10, msdr=15, iacl=2, norder=1, level=3, 
                        north=7, iredsys=1, rrctols=0.0, idroptol=1, epsrn=0.0001, 
                        hclosexmd=0.0001, mxiterxmd=200) # MODFLOW- NWT fluxtol=1e-3, 
                                                   
    print("Newton solver set up")
        
    # --- Recharge --- #
        
    # RECHARGE (RCH) PACKAGE
    if (rechargeScenario+1) == 1:
        recharge_array = recharge_array_1
    else:
        recharge_array = recharge_array_2
    
    rch = flopy.modflow.ModflowRch(mf, rech=recharge_array)
        
    print("Recharge package set up")
        
    # --- General head boundaries --- #
        
    # GENERAL HEAD BOUNDARY (GHB) PACKAGE 
    ghb = flopy.modflow.ModflowGhb(mf, stress_period_data=ghb_spd)
    print("ghd")  
        
    print("General head boundary package set up")
        
    # --- SETTING UP THE OUTPUT CONTROL --- #
        
    # OUTPUT CONTROL                   
    spd = {}
    for strsp in range(nper):
        tmstp = nstp[strsp]
        for t in range(tmstp):
            spd[strsp, t] = ['save budget', 'print budget', 'save head'] 
                                       
    oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)
        
    print("Output control package set up")
        
    # --- SETTING UP BASIC PACKAGE, IBOUND --- #
        
    bas = flopy.modflow.ModflowBas(mf, ibound=1., strt=float(sHead), 
                                       ifrefm=True, ixsec=False, hnoflo=-999.99)
        
    print("Basic package set up") 
        
    # --- LINKING FILE --- #
        
    # Set up the linking file for output from flow model to be used in transport model
    lmt = flopy.modflow.ModflowLmt(mf, output_file_name= (modelname + str('_mt3dLink.ftl')),
                                       output_file_header='extended',
                                       output_file_format='unformatted')   
                                        
    print("Link-Mass Transport package set up") 
        
    # --- NOW I NEED TO COMPILE AND WRITE ALL OF THE INPUT FILES --- #
          
    mf.write_input() # Write the model input files
        
    #print("Input written, starting to run the model at: " + str(time.ctime()))
    print("Input written")
        
    # Setting up subprocess AND RUNNING THE MODEL
    #---------------------------------------------------------------
    p = subprocess.Popen([MODFLOW_folder, modelname], shell = False)   
    p.wait()
    #---------------------------------------------------------------
    print("END: Flow model has been run") 
     

# Function to set up and run transport file

In [ ]:
def transport_model(mfmodel):   

    mt = mt3d.Mt3dms(modflowmodel=mf, modelname=modelname_mt, model_ws=this_model_folder, 
                         version=mt3d_version, namefile_ext='mtnam', exe_name=exe_name,  
                         ftlfilename=(modelname + str('_mt3dLink.ftl'))) 
               
    #------------------------------------------------------------------------------
    # BASIC TRANSPORT PACKAGE
        
    ncomp = 1         # Total number of chemical species in simulation.  
    mcomp = 1         # Total number of "mobile" species. 
    sconc= np.zeros((nlay, nrow, ncol), dtype=np.float32)  # initial conc. = 0 yrs   
    nprs = 0          # Flag indicating frequency of output. = 0: results not saved except 
                      # at end of simulation; > 0: saved at times specified by timprs; < 0: saved
                      # whenever the  number of transport steps is an even multiple of nprs.
        
    btn = mt3d.Mt3dBtn(mt, nlay=nlay, nrow=nrow, ncol=ncol, nper=nper, perlen=perlen, 
                           ncomp=ncomp, mcomp=mcomp, sconc=sconc, prsity=prsity,
                           delr=delr, delc=delc, icbund=1, ifmtcn=-1, savucn=True,
                           nprs=nprs, nprobs=1, cinact=0, ssflag='SState', laycon=1) 
        
    #------------------------------------------------------------------------------
    # ADVECTION PACKAGE
        
    adv = mt3d.Mt3dAdv(mt, mixelm=0)  
        
    # mixelm = 0 means that I am using the standard finite-difference method with upstream or 
    # central-in-stream weighting, depending on the value of NADVFD. 
    # Other options, MOC (1); MMOC (2); HMOC (3); TVD (-1). 
        
    #------------------------------------------------------------------------------
    # GENERALISED CONJUGATE GRADIENT SOLVER for MT3D-USGS
        
    gcg = mt3d.Mt3dGcg(mt, mxiter=30, iter1=50, isolve=2, accl=1, cclose=1e-06)
        
    #------------------------------------------------------------------------------
    # REACTION PACKAGE
        
    rc1 = np.zeros((nlay, nrow, ncol), dtype=np.float32)
    rc1[:, :, :] = -1/365.25
        
    isothm = 0      # 0 = no sorption.
    ireact = 100    # 100 = zeroth-order reaction option.
    rc1 = rc1       # First order reaction rate for diffolved phase of first species.
        
    rct= mt3d.Mt3dRct(mt, isothm=isothm, ireact=ireact, igetsc=0, rc1=rc1) 
        
    #------------------------------------------------------------------------------
    # SOURCE-SINK MIXING PACKAGE
        
    crch = np.zeros((nrow, ncol), dtype=np.float32) # The age of recharging water is 0
        
    itype = mt3d.Mt3dSsm.itype_dict()
       
    ssm = mt3d.Mt3dSsm(mt, crch=crch) #, stress_period_data=ssm_data) 
        
    #------------------------------------------------------------------------------
    # DISPERSION PACKAGE 
    al = 1.5 # The longitudinal dispersivity, default = 0.01

#   trpt = 0.1  #ratio of horizontal transverse: longitudinal dispersivity, default = 0.1
#   trpv = 0.01 #ratio of vertical transverse dispersivity: longitudinal dispersivity, default = 0.01
    dmcoef = 1E-4*tortuosity_factor # Effective molecular diffusion coefficient (for water in my model), default = 1E-9 m2/s
        
    # Instantiate up the Dispersion package for MT3D-USGS
    dsp = mt3d.Mt3dDsp(mt, al=al, dmcoef=dmcoef) 
        
    #----- WRITING transport MODEL ------------------------------------------------
         
    mt.write_input()
    mt.write_name_file()
        
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
    # "Manual" changes to the input files
    conc_filename_dissolved = str(modelname_mt) + ".ucn"
    conc_filename_sorbed = str(modelname_mt) + "_S.ucn"
    mass_filename = str(modelname_mt) + ".mas"
    cnf_filename = str(modelname_mt) + ".cnf"
        
    ##Add a line to the MT3D .mtnam file re-naming output UCN file + MAS,CNF
    mt_name_file = modelname_mt + str('.mtnam')
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data(binary) 201 %s \n' %(conc_filename_dissolved))
    namfile.close()
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data(binary) 301 %s \n' %(conc_filename_sorbed))
    namfile.close()
              
    namfile = open(mt_name_file, 'a')
    namfile.write('data 601 %s \n' %(mass_filename))
    namfile.close()
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data 17 %s \n' %(cnf_filename))
    namfile.close()
                
    ###For USGS need to add DRYCELL keyword to the .btn file on line 3
    mt_btn_file = modelname_mt + str('.btn')
        

    btnfile = open(mt_btn_file, "r")
    contents = btnfile.readlines()
    btnfile.close()
    contents.insert(2, "DRYCELL \n")
    btnfile = open(mt_btn_file, "w")
    contents = "".join(contents)
    btnfile.write(contents)
    btnfile.close()
        
    # ADD SSTATE FLAG manually in correct spot

    fpath = os.path.join(this_model_folder, mt_btn_file)
    with open(mt_btn_file, 'r+') as f:
        lines = f.readlines()
        f.seek(0)
        f.truncate()

        a = '%.1E' % Decimal(length_simulation)
        if length_simulation == 1:
            b = "1         1         1 S"
            c = "1         1         1         SState"
        else:
            b = str(a) + "         1         1 S"
            c = str(a) + "         1         1         SState"

        ##

        for line in lines:
            if b in line:
                            line = line.replace(b, c)
            f.write(line)
            
        # Running the transport model
        #--------------------------------
        success, buff = mt.run_model()
        print("Transport model has been run")
        #-------------------------------- 

## Saving information about the model 

I saved two per column, because I didn't know how to save only one item in the pandas DataFrame and I found this a useful way to access the details of the model run, as I ran lots of different models and wanted to readily compare output.

In [ ]:
now = datetime.datetime.now()
model_inputs = pd.DataFrame({
            "a_model_name": [str(modelName), str(modelName_mt)],
            "date_simulation": [str(now.strftime("%Y-%m-%d %H:%M")), str(now.strftime("%Y-%m-%d %H:%M"))],
            "hk_aquifer": [hk_aquifer, hk_aquifer],
            "hk_barrier": [hk_barrier, hk_barrier],
            "model_spacing": [delr, delr],
            "number_layers": [nlay, nlay],
            "length_x": [Lx, Lx],
            "length_y": [Ly, Ly],
            "thickness": [ztop, ztop],
            "starting_head": [sHead, sHead],
            "specific_yield": [sy, sy],
            "specific_storage": [ss, ss],
            "porosity": [prsity, prsity],
            "recharge_total_per_y": [total_recharge_per_y, total_recharge_per_y], 
            "length_simulation": [length_simulation, length_simulation],
            "run_time": [run_time, run_time],
            "superComp": [am_I_running_on_supercomputer, am_I_running_on_supercomputer]
                                        })

fileName = os.path.join(this_model_folder, 'model_inputs.csv')
model_inputs.to_csv(fileName, encoding='utf-8')